In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/lit/lit_lit/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import deslib
from deslib.dcs import OLA, MLA
from deslib.des import METADES, KNORAU, DESMI, DESP
from deslib.des.knora_e import KNORAE
from deslib.static import Oracle
from sklearn.metrics import accuracy_score
import glob
import numpy as np
from numpy import mean
import pickle
from glob import glob
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
import sklearn.metrics as metrics
from sklearn import metrics
from sklearn.metrics import fbeta_score, make_scorer
import sklearn
from numpy import mean

dict_final_results = {}
dict_standard_deviation = {}
for files in glob('/content/drive/MyDrive/lit/lit_lit/reserve/processed_8/*'):
  print(files)

/content/drive/MyDrive/lit/lit_lit/reserve/processed_8/Adult_dir


In [ ]:
# import deslib
# from deslib.dcs import OLA, MLA
# from deslib.des import METADES, KNORAU, DESMI, DESP
# from deslib.des.knora_e import KNORAE
# from deslib.static import Oracle
# from sklearn.metrics import accuracy_score
# import glob
# import numpy as np
# from numpy import mean
# import pickle
# from glob import glob
# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import auc
# import sklearn.metrics as metrics
# from sklearn import metrics
# from sklearn.metrics import fbeta_score, make_scorer
# import sklearn
# from numpy import mean


# dict_final_results = {}
# dict_standard_deviation = {}
# for files in glob('/content/drive/MyDrive/lit/lit_lit/reserve/processed_8/*'):
#     print(files)
#     standard_deviation = []
#     knorae_results = []
#     accuracy = []
#     for items in glob("{}/*.*".format(files)):
#         print(items)
#         data = np.load(items, allow_pickle=True)
#         input_list = data.tolist()  # How to get x_train
#         X_train = input_list['FrameStack'][0]
#         X_test = input_list['FrameStack'][1]
#         y_train = input_list['FrameStack'][2]
#         y_test = input_list['FrameStack'][3]
#         X_dsel = input_list['FrameStack'][4]
#         y_dsel = input_list['FrameStack'][5]

#         ###file_name = files.split("\\")[1]
#         # print(file_name)
#         ###item_name = items.split("\\")[2].split(".")[0].split("_")[2]
#         # print(item_name)
#         ###ss = file_name.split("_")[0]
#         # print(ss)
#         # model_load = glob.glob("models_f/{}/split_{}/bagging_with_decision_tree.pkl".format(file_name, item_name))[0]
#         # model_load = glob.glob("models_f/{}/split_{}/bagging_with_perceptron.pkl".format(file_name, item_name))[0]
#         # model_load = glob.glob("models_f/{}/split_{}/boosting_with_decision_tree.pkl".format(file_name, item_name))[0]
#         # model_load = glob.glob("models_f/{}/split_{}/boosting_with_perceptron.pkl".format(file_name, item_name))[0]
#         # model_load = glob.glob("models_f/{}/split_{}/random_forest.pkl".format(file_name, item_name))[0]
#         # model_load = glob.glob("Models_FLT_f/{}/{}_{}.pkl".format(file_name,ss, item_name))[0]
#         # model_load = glob.glob("models_f/{}/split_{}/bagging_with_decision_tree.pkl".format(file_name, item_name))[0]
#         model_load = glob("/content/drive/MyDrive/lit/lit_lit/input_space_model0.pkl")   #input_space_model0.pkl

#         x_merge = np.append(X_train, X_dsel, axis=0)
#         y_merge = np.append(y_train, y_dsel, axis=0)

#         pool_classifiers = model_load
#         # method = KNORAE
#         # method = MLA
#         # method = OLA
#         # method = METADES
#         # method = KNORAU
#         method = DESMI
#         # method = DESP

#         ensemble = method(pool_classifiers)
#         # ensemble.fit(x_train, y_train)
#         ensemble.fit(x_merge, y_merge)
#         # Predict new examples:
#         yh = ensemble.predict(X_test)
#         acc = accuracy_score(y_test, yh)
#         print(acc)
#         knorae_results.append(acc)

#     dict_final_results["{}".format(files)] = '{:.4f}'.format(mean(knorae_results))
#     print(dict_final_results)
#     np.std(knorae_results, dtype=np.float64)
#     standard_deviation.append(np.std(knorae_results, dtype=np.float64))
#     dict_standard_deviation["{}".format(files)] = '{:.4f}'.format(mean(standard_deviation))

# print(dict_final_results.values())
# print(dict_standard_deviation.values())

In [ ]:
import uuid
import numpy as np
import tensorflow as tf
import six.moves.cPickle as pickle
from six import add_metaclass
from abc import ABCMeta, abstractmethod, abstractproperty
# from utils import *

"""
Object-oriented class for handling neural networks implemented
in Tensorflow.
"""
@add_metaclass(ABCMeta)
class NeuralNetwork():
  def __init__(self, name=None, dtype=tf.float32, **kwargs):
    self.vals = None # Holds the trained weights of the network
    self.name = (name or str(uuid.uuid4())) # Tensorflow variable scope
    self.dtype = dtype
    self.setup_model(**kwargs)
    assert(hasattr(self, 'X'))
    assert(hasattr(self, 'y'))
    assert(hasattr(self, 'logits'))

  def setup_model(self, X=None, y=None, **kw):
    """Defines common placeholders, then calls rebuild_model"""
    with tf.name_scope(self.name):

      self.X = tf.compat.v1.placeholder(self.dtype, self.x_shape , name="X") if X is None else X
      self.y = tf.compat.v1.placeholder(self.dtype, self.y_shape, name="y") if y is None else y
      self.sample_weight = tf.compat.v1.placeholder(self.dtype, [None], name="sample_weight")
      self.is_train = tf.compat.v1.placeholder_with_default(
          tf.constant(False, dtype=tf.bool), shape=(), name="is_train")
    self.model = self.rebuild_model(self.X, **kw)
    self.recompute_vars()

  def rebuild_model(self, X, reuse=None, **kw):
    """Override this in subclasses. Define Tensorflow operations and return
    list whose last entry is logits."""

  @property
  def logits(self):
    return self.model[-1]

  @abstractproperty
  def x_shape(self):
    """Specify the shape of X; for MNIST, this could be [None, 784]"""

  @abstractproperty
  def y_shape(self):
    """Specify the shape of y; for MNIST, this would be [None, 10]"""

  @property
  def num_features(self):
    return np.product(self.x_shape[1:])

  @property
  def num_classes(self):
      
    return np.product(self.y_shape[1:])

  @property
  def trainable_vars(self):
    """Return this model's trainable variables"""
    return [v for v in tf.compat.v1.trainable_variables() if v in self.vars]

  def input_grad(self, f):
    """Helper to take input gradients"""
    return tf.gradients(f, self.X)[0]

  def cross_entropy_with(self, y):
    """Compute sample-weighted cross-entropy classification loss"""
    w = self.sample_weight / tf.reduce_sum(self.sample_weight)
    return tf.reduce_sum(tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits, labels=y) * w)

  @cachedproperty
  def preds(self):
    """Tensorflow operation to return predicted labels."""
    return tf.argmax(self.logits, axis=1)

  @cachedproperty
  def probs(self):
    """Tensorflow operation to return predicted label probabilities."""
    return tf.nn.softmax(self.logits)

  @cachedproperty
  def logps(self):
    """Tensorflow operation to return predicted label log-probabilities."""
    return self.logits - tf.reduce_logsumexp(self.logits, 1, keepdims=True)

  @cachedproperty
  def grad_sum_logps(self):
    """Tensorflow operation returning gradient of the sum of log-probabilities.
    Can be used as the gradient for LIT for multi-class classification (doesn't
    require labels)."""
    return self.input_grad(self.logps)

  @cachedproperty
  def l2_weights(self):
    """Tensorflow operation returning sum of squared weight values"""
    return tf.add_n([l2_loss(v) for v in self.trainable_vars])

  @cachedproperty
  def cross_entropy(self):
    """Tensorflow operation returning classification cross-entropy"""
    return self.cross_entropy_with(self.y)

  @cachedproperty
  def cross_entropy_input_gradients(self):
    """Tensorflow operation returning gradient of the loss. Can be used as the
    gradient for LIT for multi-class classification but does require labels."""
    return self.input_grad(self.cross_entropy)

  @cachedproperty
  def predicted_logit_input_gradients(self):
    """Tensorflow operation returning gradient of the predicted log-odds.
    Mostly useful for visualization rather than training."""
    return self.input_grad(self.logits * self.y)

  @cachedproperty
  def binary_logits(self):
    """Tensorflow operation returning the actual predicted log-odds (binary
    only)."""
    assert(self.num_classes == 2)
    return self.logps[:,1] - self.logps[:,0]

  @cachedproperty
  def binary_logit_input_gradients(self):
    """Tensorflow operation returning gradient of the predicted binary
    log-odds. This is what we use for LIT in binary classification."""
    return self.input_grad(self.binary_logits)

  @cachedproperty
  def accuracy(self):
    """Tensorflow operation returning classification accuracy."""
    return tf.reduce_mean(tf.cast(tf.equal(self.preds, tf.argmax(self.y, 1)), dtype=tf.float32))

  def score(self, X, y, **kw):
    """Compute classification accuracy for numpy inputs and labels."""
    if len(y.shape) == 2:
      return np.mean(self.predict(X, **kw) == np.argmax(y, 1))
    else:
      return np.mean(self.predict(X, **kw) == y)

  def predict(self, X, **kw):
    """Compute predictions for numpy inputs."""
    with tf.Session() as sess:
      self.init(sess)
      return self.batch_eval(sess, self.preds, X, **kw)

  def predict_logits(self, X, **kw):
    """Compute raw logits for numpy inputs."""
    with tf.compat.v1.Session() as sess:
      self.init(sess)
      return self.batch_eval(sess, self.logits, X, **kw)

  def predict_binary_logodds(self, X, **kw):
    """Compute predicted binary log-odds for numpy inputs."""
    with tf.compat.v1.Session() as sess:
      self.init(sess)
      return self.batch_eval(sess, self.binary_logits, X, **kw)

  def predict_proba(self, X, **kw):
    """Compute predicted probabilities for numpy inputs."""
    with tf.compat.v1.Session() as sess:
      self.init(sess)
      return self.batch_eval(sess, self.probs, X, **kw)

  def batch_eval(self, sess, quantity, X, n=256):
    """Internal helper to batch computations (prevents memory issues)"""
    vals = sess.run(quantity, feed_dict={ self.X: X[:n] })
    stack = np.vstack if len(vals.shape) > 1 else np.hstack
    for i in range(n, len(X), n):
      vals = stack((vals, sess.run(quantity, feed_dict={ self.X: X[i:i+n] })))
    return vals

  def input_gradients(self, X, y=None, n=256, **kw):
    """Computes different kinds of input gradients for inputs (and optionally
    labels). See input_gradients_ for details."""
    with tf.compat.v1.Session() as sess:
      self.init(sess)
      return self.batch_input_gradients_(sess, X, y, n, **kw)

  def batch_input_gradients_(self, sess, X, y=None, n=256, **kw):
    yy = y[:n] if y is not None and not isint(y) else y
    grads = self.input_gradients_(sess, X[:n], yy, **kw)
    for i in range(n, len(X), n):
      yy = y[i:i+n] if y is not None and not isint(y) else y
      grads = np.vstack((grads,
        self.input_gradients_(sess, X[i:i+n], yy, **kw)))
    return grads

  def input_gradients_(self, sess, X, y=None, logits=False, quantity=None):
    if quantity is not None:
      return sess.run(quantity, feed_dict={ self.X: X })
    if y is None:
      return sess.run(self.grad_sum_logps, feed_dict={ self.X: X })
    elif logits and self.num_classes == 2:
      return sess.run(self.binary_logit_input_gradients, feed_dict={ self.X: X })
    elif isint(y):
      y = onehot(np.array([y]*len(X)), self.num_classes)
    feed = { self.X: X, self.y: y, self.sample_weight: np.ones(len(X)) }
    if logits:
      return sess.run(self.predicted_logit_input_gradients, feed_dict=feed)
    else:
      return sess.run(self.cross_entropy_input_gradients, feed_dict=feed)

  def loss_function(self, l2_weights=0., **kw):
    """Construct the loss function Tensorflow op given hyperparameters."""
    log_likelihood = self.cross_entropy
    if l2_weights > 0:
      log_prior = l2_weights * self.l2_weights
    else:
      log_prior = 0
    return log_likelihood + log_prior

  def fit(self, X, y, loss_fn=None, init=False, sample_weight=None, **kw):
    """Fit the neural network on the particular dataset."""
    if loss_fn is None:
      loss_fn = self.loss_function(**kw)
    if len(y.shape) == 1:
      y = onehot(y, self.num_classes)
    if sample_weight is None:
      sample_weight = np.ones(len(X))
    ops = { 'xent': self.cross_entropy, 'loss': loss_fn, 'accu': self.accuracy }
    batches = train_batches([self], X, y, sample_weight=sample_weight, **kw)
    with tf.compat.v1.Session() as sess:
      if init: self.init(sess)
      minimize(sess, loss_fn, batches, ops, **kw)
      self.vals = [v.eval() for v in self.vars]

  def recompute_vars(self):
    """Determine which Tensorflow variables are associated with this
    network."""
    self.vars = tf.compat.v1.get_default_graph().get_collection(
        tf.compat.v1.GraphKeys.GLOBAL_VARIABLES, scope=self.name)

  def init(self, sess):
    """Prepare this network to be used in a Tensorflow session."""
    if self.vals is None:
      sess.run(tf.global_variables_initializer())
    else:
      for var, val in zip(self.vars, self.vals):
        sess.run(var.assign(val))

  def save(self, filename):
    """Save the weights of the network to a pickle file."""
    with open(filename, 'wb') as f:
      pickle.dump(self.vals, f)

  def load(self, filename):
    """Load the weights of the network from a pickle file."""
    with open(filename, 'rb') as f:
      self.vals = pickle.load(f)

In [ ]:
from __future__ import print_function
import six
import time
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split

def l1_loss(x):
  return tf.reduce_sum(tf.abs(x))

def l2_loss(x):
  return tf.nn.l2_loss(x)

class cachedproperty(object):
  """Simplified https://github.com/pydanny/cached-property"""
  def __init__(self, function):
    self.__doc__ = getattr(function, '__doc__')
    self.function = function

  def __get__(self, instance, klass):
    if instance is None: return self
    value = instance.__dict__[self.function.__name__] = self.function(instance)
    return value

def isint(x):
  return isinstance(x, (int, np.int32, np.int64))

def onehot(Y, K=None):
  if K is None:
    K = np.unique(Y)
  elif isint(K):
    K = list(range(K))
  data = np.array([[y == k for k in K] for y in Y]).astype(int)
  return data

def minibatch_indexes(lenX, batch_size=256, num_epochs=50, **kw):
  n = int(np.ceil(lenX / batch_size))
  for epoch in range(num_epochs):
    for batch in range(n):
      i = epoch*n + batch
      
      yield i, epoch, slice((i%n)*batch_size, ((i%n)+1)*batch_size)

def train_feed(idx, models, **kw):
  """Convert a set of models, a set of indexes, and numpy arrays given by the
  keyword arguments to a set of feed dictionaries for each model."""
  feed = {}
  for m in models:
   
    feed[m.is_train] = True
    for dictionary in [kw, kw.get('feed_dict', {})]:
      for key, val in six.iteritems(dictionary):
        attr = getattr(m, key) if isinstance(key, str) and hasattr(m, key) else key
        
        if type(attr) == type(m.X):
          if len(attr.shape) >= 1:
            if attr.shape[0] is None:
              feed[attr] = val[idx]
  
  return feed

def train_batches(models, X, y, **kw):
  for i, epoch, idx in minibatch_indexes(len(X), **kw):
    yield i, epoch, train_feed(idx, models, X=X, y=y, **kw)

def reinitialize_variables(sess):
  """Construct a Tensorflow operation to initialize any variables in its graph
  which are not already initialized."""
  uninitialized_vars = []
  for var in tf.compat.v1.global_variables():
    try:
      sess.run(var)
    except tf.errors.FailedPreconditionError:
      uninitialized_vars.append(var)
  return tf.compat.v1.variables_initializer(uninitialized_vars)

def minimize(sess, loss_fn, batches, operations={}, learning_rate=0.001, print_every=None, var_list=None, **kw):
  """Minimize a loss function over the provided batches of data, possibly
  printing progress."""
  optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
  train_op = optimizer.minimize(loss_fn, var_list=var_list)
  op_keys = sorted(list(operations.keys()))
  ops = [train_op] + [operations[k] for k in op_keys]
  t = time.time()
  sess.run(reinitialize_variables(sess))
  for i, epoch, batch in batches:

    results = sess.run(ops, feed_dict=batch)
    if print_every and i % print_every == 0:
      s = 'Batch {}, epoch {}, time {:.1f}s'.format(i, epoch, time.time() - t)
      for j,k in enumerate(op_keys, 1):
        s += ', {} {:.4f}'.format(k, results[j])
      print(s)

def tt_split(X, y, test_size=0.2):
  return train_test_split(X, y, test_size=test_size, stratify=y)

def elemwise_sq_cos_sim(v, w, eps=1e-8):
  assert(len(v.shape) == 2)
  assert(len(w.shape) == 2)
  num = np.sum(v*w, axis=1)**2
  den = np.sum(v*v, axis=1) * np.sum(w*w, axis=1)
  return num / (den + eps)

def yules_q_statistic(e1, e2, y_test):
  n = len(y_test)
  n00 = len(e1.intersection(e2))
  n01 = len(e1.difference(e2))
  n10 = len(e2.difference(e1))
  n11 = n - len(e1.union(e2))
  assert(n00+n01+n10+n11 == n)
  numer = n11*n00 - n01*n10
  denom = n11*n00 + n01*n10
  if numer == 0:
    return 0
  else:
    return numer / float(denom)

def disagreement_measure(e1, e2, y_test):
  n = len(y_test)
  n01 = len(e1.difference(e2))
  n10 = len(e2.difference(e1))
  return (n01 + n10) / n

def scoring_fun(y_pred, y_true):
  if len(y_true.shape) == 1:
    #assert(y_true.max() == 1) # binary
    if len(y_pred.shape) == 1:
      preds = y_pred
    else:
      preds = y_pred[:,1]
    
    return accuracy_fun(y_pred, y_true)
    #return roc_auc_score(y_true, preds)
  else:
    return accuracy_fun(y_pred, y_true)

def accuracy_fun(y_pred, y_true):
  if len(y_true.shape) == 1:
    #assert(y_true.max() == 1) # binary
    if len(y_pred.shape) == 1:
      preds = (y_pred > 0.5).astype(int)
    else:
      preds = np.argmax(y_pred, axis=1)
    return np.mean(y_true == preds)
  else:
    return np.mean(np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1))

def error_masks(y_pred, y_true):
  if len(y_true.shape) == 1:
    #assert(y_true.max() == 1) # binary
    if len(y_pred.shape) == 1:
      preds = (y_pred > 0.5).astype(int)
    else:
      preds = np.argmax(y_pred, axis=1)
    return (preds != y_true).astype(int)
  else:
    return (np.argmax(y_true, axis=1) != np.argmax(y_pred, axis=1)).astype(int)

In [ ]:
class Net(NeuralNetwork): 
  @property
  def x_shape(self): return [None, 24]
  @property
  def y_shape(self): return [None, 2]
  def rebuild_model(self, X, **_):
    L0 = X
    L1 = tf.compat.v1.layers.dense(L0, 256, name=self.name+'/L1', activation=tf.nn.softplus)
    L2 = tf.compat.v1.layers.dense(L1, 256, name=self.name+'/L2', activation=tf.nn.softplus)
    L3 = tf.compat.v1.layers.dense(L2,  2, name=self.name+'/L3', activation=None)
    return [L1, L2, L3]

In [ ]:
import deslib
from deslib.dcs import OLA, MLA
from deslib.des import METADES, KNORAU, DESMI, DESP
from deslib.des.knora_e import KNORAE
from deslib.static import Oracle
from sklearn.metrics import accuracy_score
import glob
import numpy as np
from numpy import mean
import pickle
from glob import glob
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
import sklearn.metrics as metrics
from sklearn import metrics
from sklearn.metrics import fbeta_score, make_scorer
import sklearn
from numpy import mean


dict_final_results = {}
dict_standard_deviation = {}
for files in glob('/content/drive/MyDrive/lit/lit_lit/reserve/processed_8/*'):
    print(files)
    standard_deviation = []
    knorae_results = []
    accuracy = []
    for items in glob("{}/*.*".format(files)):
        print(items)
        data = np.load(items, allow_pickle=True)
        input_list = data.tolist()  # How to get x_train
        X_train = input_list['FrameStack'][0]
        X_test = input_list['FrameStack'][1]
        y_train = input_list['FrameStack'][2]
        y_test = input_list['FrameStack'][3]
        X_dsel = input_list['FrameStack'][4]
        y_dsel = input_list['FrameStack'][5]

        ###file_name = files.split("\\")[1]
        # print(file_name)
        ###item_name = items.split("\\")[2].split(".")[0].split("_")[2]
        # print(item_name)
        ###ss = file_name.split("_")[0]
        # print(ss)
        # model_load = glob.glob("models_f/{}/split_{}/bagging_with_decision_tree.pkl".format(file_name, item_name))[0]
        # model_load = glob.glob("models_f/{}/split_{}/bagging_with_perceptron.pkl".format(file_name, item_name))[0]
        # model_load = glob.glob("models_f/{}/split_{}/boosting_with_decision_tree.pkl".format(file_name, item_name))[0]
        # model_load = glob.glob("models_f/{}/split_{}/boosting_with_perceptron.pkl".format(file_name, item_name))[0]
        # model_load = glob.glob("models_f/{}/split_{}/random_forest.pkl".format(file_name, item_name))[0]
        # model_load = glob.glob("Models_FLT_f/{}/{}_{}.pkl".format(file_name,ss, item_name))[0]
        # model_load = glob.glob("models_f/{}/split_{}/bagging_with_decision_tree.pkl".format(file_name, item_name))[0]
        input_space_models = [Net() for _ in range(2)]
        for i, model in enumerate(input_space_models):
          model.load("/content/drive/MyDrive/lit/lit_lit/input_space_model"+str(i)+".pkl")   #input_space_model0.pkl

          x_merge = np.append(X_train, X_dsel, axis=0)
          y_merge = np.append(y_train, y_dsel, axis=0)

          pool_classifiers = model_load
          method = KNORAE
          # method = MLA
          # method = OLA
          # method = METADES
          # method = KNORAU
          # method = DESMI
          # method = DESP

          ensemble = method(pool_classifiers)
          # ensemble.fit(X_train, y_train)
          ensemble.fit(x_merge, y_merge)
          # Predict new examples:
          yh = ensemble.predict(X_test)
          acc = accuracy_score(y_test, yh)
          print(acc)
          knorae_results.append(acc)

    dict_final_results["{}".format(files)] = '{:.4f}'.format(mean(knorae_results))
    print(dict_final_results)
    np.std(knorae_results, dtype=np.float64)
    standard_deviation.append(np.std(knorae_results, dtype=np.float64))
    dict_standard_deviation["{}".format(files)] = '{:.4f}'.format(mean(standard_deviation))

print(dict_final_results.values())
print(dict_standard_deviation.values())

/content/drive/MyDrive/lit/lit_lit/reserve/processed_8/Adult_dir
/content/drive/MyDrive/lit/lit_lit/reserve/processed_8/Adult_dir/processed_Adult_1.npy


RuntimeError: ignored

In [ ]:
# bedone save
from glob import glob
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
import numpy as np
import sklearn.metrics as metrics
from sklearn import metrics
from sklearn.metrics import fbeta_score, make_scorer
import sklearn
from numpy import mean

dict_results = {}
dict_sd_pc = {}
for files in glob('//content/drive/MyDrive/lit/lit_lit/reserve/processed_8/*'):
    print(files)
    pc = []
    sd_pc = []
    for items in glob("{}/*.*".format(files)):
        print(items)
        item_name = items.split(".")[0]
        print(item_name)
        ss = item_name.split("_")[4] + item_name.split("_")[5]
        print(ss)
        data = np.load(items, allow_pickle=True)
        input_list = data.tolist()  # How to get x_train
        X_train = input_list['FrameStack'][0]
        X_test = input_list['FrameStack'][1]
        y_train = input_list['FrameStack'][2]
        y_test = input_list['FrameStack'][3]
        X_val = input_list['FrameStack'][4]
        y_val = input_list['FrameStack'][5]
        print(len(X_train))

        y_shape = max(y_train)+1
        class Net(NeuralNetwork): 
          @property
          def x_shape(self): return [None, X_train.shape[1]]
          @property
          def y_shape(self): return [None, y_shape]
          def rebuild_model(self, X, **_):
            L0 = X
            L1 = tf.compat.v1.layers.dense(L0, 256, name=self.name+'/L1', activation=tf.nn.softplus)
            L2 = tf.compat.v1.layers.dense(L1, 256, name=self.name+'/L2', activation=tf.nn.softplus)
            L3 = tf.compat.v1.layers.dense(L2, y_shape , name=self.name+'/L3', activation=None)
            return [L1, L2, L3]

      
        tf.compat.v1.reset_default_graph()

        print("train restarts")
        random_restart_models = train_restart_models(Net, 2, X_train, y_train, num_epochs=100, print_every=50)

        print("train input-space LIT")
        input_space_models = train_diverse_models(Net, 2, X_train, y_train, num_epochs=100, print_every=50)
        all_model_probs = np.array([ model.predict_proba(X_test)[:,1] for model in input_space_models ])
        average_probs = all_model_probs.mean(axis=0) # average predicted probability
        variance_probs = all_model_probs.var(axis=0) # variance = measure of uncertainty
        label_predictions = (average_probs >= 0.5).astype(int) # threshold average to get label prediction
        accuracy = np.mean(label_predictions == y_test)
        print(items)
        print(accuracy)
#         pc.append(accuracy)
#         dict_results["{}".format(files)] = '{:.4f}'.format(mean(pc))
#         np.std(pc, dtype=np.float64)
#         sd_pc.append(np.std(pc, dtype=np.float64))

#     dict_sd_pc["{}".format(files)] = '{:.4f}'.format(mean(sd_pc))
# print(dict_results)
# print(dict_results.values())
# print(dict_sd_pc.values())